# Wstęp do Sztucznej Inteligencji - rok akademicki 2020/2021

Przed rozpoczęciem pracy z notatnikiem zmień jego nazwę zgodnie z wzorem: `NrAlbumu_Nazwisko_Imie_PoprzedniaNazwa`.

Przed wysłaniem notatnika upewnij się, że rozwiązałeś wszystkie zadania/ćwiczenia.

# Temat: Optymalizacja globalna: Prosty algorytm genetyczny cz. I
Głownym celem zajęć poświęconych alorytmom genetycznym jest stworzenie od podstaw (implentacja) prostego algorytmu genetycznego i późniejsze wykorzystanie go do rozwiązania przykładowych zadań optymalizacji globalnej.

W tym notatniku będą Państwo mieli za zadanie zaimplementować pięć elementów (pięć funkcji) wchodzących w skład procedury algorytmu genetycznego. 

## Import biblioteki numpy
Wszystkie funkcje należy tworzyć z wykorzystaniem biblioteki `numpy`.

In [8]:
import numpy as np

## Przykładowa funkcja celu
Zadanie optymalizacji w wielu przypadakach sprowadza się optymalizacji odpowiednio sformułowanej funkcj, tzw. funkcji celu. Poniżej przykładowa prosta funkcja umożliwiająca testowanie zaimplemetowanych funkcji.  

In [10]:
# testowa funkcja celu
# x - jednowymiarowa tablica ndarray
def obj_func(x):
    return (x**2).sum()

Przykład wywołania.

Wektor `x` to tak zwany osobnik, czyli jedno z możliwych (dopuszczalnych) rozwiązań.  

In [11]:
# przykład wywołania
x = np.array([0, 8, 0, 0, 0])
print(obj_func(x))

64


Zadanie optymalizacji można sformułować jako zadanie poszukiwania minimum (minimalizacja) bądź maksimum (maksymalizacja) funkcji. 

Zaznaczmy, że rozwiązaniem problemu optymalizacji jest podanie nie tylko jaka jest wartość optymalna, ale również (a może nawet przede wszystkim) dla jakich wartości $x$ funkcja osiąga to optimum.

Przykład: Znajdź minimum funkcji $f(x)=x^2$ w przedziale $[-1, 1]$.

Rozwiązanie można sformułować następnująco: w przedziale $[-1, 1]$ funkcja $f(x)=x^2$ osiąga minimum równe $0$ dla $x=0$.

Zwróć uwagę, że powyższe zadanie można zdefiniować jako zadanie maksymalizacji:

Przykład: Znajdź maksimum funkcji $f(x)=-x^2$ w przedziale $[-1, 1]$.

Rozwiązanie jest takie samo, tzn. dla $x=0$ podana funkcja osiąga wartość $0$ (tym razem jest to maksimum). 

Zatem, implementując nasz aglorytm genetyczny do szukania maksimum funkcji, będziemy go w stanie użyć do szukania minimum danej funkcji, jeśli weźmiemy oryginalną funkcję i pomnożymy ją przez $-1$.


## Liczba potrzebnych bitów
W prosty algorytmie genetycznym wykorzystywane jest kodowanie binarne osobnika, tzn. każda z wartości rzeczywistych wektora `x` reprezentowana jest przez ciąg bitów (zer i jedynek).

Pierwszym krokiem zatem jest określenie ile potrzeba bitów aby móc zakodować wszystkie dopuszczalne rozwiązania z zadaną dokładnością.

### Zadanie 1
Zaimplementować metodę obliczającą ilość bitów potrzebnych do zakodowania liczby rzeczywistej z przedziału `[a, b]` z zadanym krokiem `dx`. Metoda ta powinna zwracać również nowy dokładniejszy krok `dx`.

Należy zatem na podstwie kroku `dx` oraz końców przedziału `a` i `b` określić ile liczb całkowitych będzie trzeba zakodować w postaci binarnej. Nasępnie dobrać najmniejszą liczbę bitów pozwalającą na zakodowanie tylu liczb.

__Przykład:__ `a=0`, `b=1`, `dx=0.1`

W przedziale `[0, 1]` z krokiem `0.1` mieści się 11 liczb (włącznie z końcami przedziału), zatem potrzebna liczba bitów to 4 bo na 4 bitach zakodujemy 16 (od 0 do 15) liczb a na 3 już tylko 8 (za mało). 

Ponieważ na 4 bitach zakodujemy 16 liczb to przy niezmienionym kroku liczba:
- `0000` odpowiada liczbie całkowitej 0 ($i$), a rzeczywistej 0.0 (wzór: $i*dx+a$)
- `1111` odpowiada liczbie całkowitej 15 ($i$), a rzeczywistej 1.5 (wzór: $i*dx+a$)

Jak widać liczba `1111` po rozkodowaniu wykracza poza dopuszczalny podział. Należy zatem zaktualizować krok `dx` tak aby `1111` odpowiadało dokładnie wartości `b`.

Argumenty funkcji:
- `a` - początek przedziału, liczba rzeczywista.
- `b` - koniec przedziału, liczba rzeczywista.
- `dx` - krok, dokładność kodowania, liczba rzeczywista.
- `B` - liczba bitów, liczba całkowita.
- `dx_new` - nowy dokładniejszy krok, liczba rzeczywista.

In [69]:
from math import fabs
def nbits(a, b, dx):
    ### TWÓJ KOD TUTAJ
    num = np.arange(a, b+dx, dx)
    B=0
    
    while(1):
        if 2**B > num.size:
            break
        else:
            B=B+1
            continue
    
    dx_new = (b-a)/((2**B) - 1)
    
    return B,dx_new

nbits(0,1,0.1) #widzimy bezbledne działanie funkcji





(4, 0.06666666666666667)

In [70]:
# jeśli poniższe nie rzuca wyjątku to znaczy, że testy przeszły ale nie musi oznaczać, że wszystko jest dobrze
np.testing.assert_almost_equal(nbits(0, 1, 0.1)[0], 4, decimal=6)
np.testing.assert_almost_equal(nbits(0, 1, 0.1)[1], 0.06666666666666667, decimal=6)
np.testing.assert_almost_equal(nbits(-2.4, 3.1, 0.01)[0], 10, decimal=6)
np.testing.assert_almost_equal(nbits(-2.4, 3.1, 0.01)[1], 0.005376344086021506, decimal=6)

## Populacja początkowa
Algorytm genetyczny jest algorytmem działającym na pewnej populacji osobników (początkowo losowej), którą to poddaje się tzw. operacjom genetycznym.


### Zadanie 2
Zaimplementować metodę generującą początkową populację zakodowanych osobników (binarną). Metoda ta powinna zwracać obiekt typu `ndarray`. Użyj metody `np.random.randint`.

Jest to po prostu dwuwymiarowa tablica, gdzie pierwszy wymiar to liczba osobników, a drugi liczba zmiennych w osobniku razy liczba bitów na każdą z nich.

Uwaga: W naszej implementacji algortmu genetycznego, dla uproszczenia przyjmiemy, że każdą zmienną rzeczywistą kodować będziemy za pomocą takiej samej liczby bitów. Ułatwia to implementację, jednak warto pamiętać, że w rzeczywistych problemach może to nie wystarczyć. Może istnieć potrzeba dokładniejszego reprezentowania pewnej zmiennej rzeczywistej (wykorzystując większą liczbę bitów) niż innej. Problem ten pojawia się również, jeśli wszystkie zmienne chcemy reprezentować z tą samą dokładności, ale zakresy ich wartości różnią się - aby osiągnąć tę samą dokładność przy większym zakresie wartości musimy użyć większej liczby bitów. 

Warto zwrócić uwagę, że szukając rozwiązania jako liczby rzeczywistej ale stosująć kodowanie binarne, z góry wiemy, że pewnych wartości nie będziemy w stanie reprezentować wykorzystując przyjętą liczbę bitów. 

Przykładowo, mamy za zadanie znalezienie minimum funkcji

$f(x) = x^2$

w przedziale $[-1, 1]$.

Wiadomo, że minimum tej funkcji jest w $x=0$ i wynosi $f(0)=0^2=0$. Jeśli jednak nasze rozwiązania reprezentujemy za pomocą 2 bitów, nie jesteśmy w stanie reprezentować wartości zero. Uruchom poniższy przykład.



In [ ]:
import numpy as np

def fun(x):
    return x*x

a = -1.0
b = 1.0

bits_num = 2

print('{0:>6} | {1:>15} | {2:>15}'.format('binary', 'decoded', 'wartosc funkcji'))
for i in range(2**bits_num):
    binary_solution = bin(i)[2:].zfill(bits_num)
    decoded_solution = a + i*(b-a)/(2**bits_num-1)
    print('{0:>6} | {1:15.10f} | {2:15.10f}'.format(binary_solution, decoded_solution, fun(decoded_solution)))

binary |         decoded | wartosc funkcji
    00 |   -1.0000000000 |    1.0000000000
    01 |   -0.3333333333 |    0.1111111111
    10 |    0.3333333333 |    0.1111111111
    11 |    1.0000000000 |    1.0000000000


Jeśli użyjemy trzech bitów, również nie jesteśmy w stanie reprezentować zera, jednak najlepsze (najbliższe 0) rozwiązanie, jakie jesteśmy w stanie reprezentować jest bliżej rzeczywistego.

In [35]:
import numpy as np

def fun(x):
    return x*x

a = -1.0
b = 1.0

bits_num = 3

print('{0:>3} | {1:>15} | {2:>15}'.format('binary', 'decoded', 'wartosc funkcji'))
for i in range(2**bits_num):
    binary_solution = bin(i)[2:].zfill(bits_num)
    decoded_solution = a + i*(b-a)/(2**bits_num-1)
    print('{0:>6} | {1:15.10f} | {2:15.10f}'.format(binary_solution, decoded_solution, fun(decoded_solution)))

binary |         decoded | wartosc funkcji
   000 |   -1.0000000000 |    1.0000000000
   001 |   -0.7142857143 |    0.5102040816
   010 |   -0.4285714286 |    0.1836734694
   011 |   -0.1428571429 |    0.0204081633
   100 |    0.1428571429 |    0.0204081633
   101 |    0.4285714286 |    0.1836734694
   110 |    0.7142857143 |    0.5102040816
   111 |    1.0000000000 |    1.0000000000


Warto zaznaczyć jeszcze jedną rzecz. Typy float i double również są reprezentowane w komputerach binarnie i mają swoje ograniczenia - niektórych wartości nie da się reprezentować. Zatem zwiększanie liczby bitów w naszej implementacji również ma sens tylko do pewnego momentu.

Zaimplementuj funkcję generującą początkową populację zakodowanych osobników (binarną). Metoda ta powinna zwracać obiekt typu ndarray. Użyj metody np.random.randint.

Argumenty funkcji:
- `P` - liczba osobników, liczba całkowita.
- `N` - liczba zmiennych, liczba całkowita.
- `B` - liczba bitów na każdą ze zmiennych, liczba całkowita.
- `pop` - populacja zakodowanych osobników, tablica `ndarray`.

In [71]:
def gen_population(P, N, B):
    ### TWÓJ KOD TUTAJ
    ### Każdy wiersz to jeden osobnik, a więc będzie P wierszy Kolumn będzie N*B, bo każda zmienna jest zapisana na B-bitach
    pop = np.random.randint(2, size=(P, N*B))
    return pop
gen_population(1, 1, 5)

array([[1, 1, 1, 1, 0]])

In [72]:
# jeśli poniższe nie rzuca wyjątku to znaczy, że testy przeszły ale nie musi oznaczać, że wszystko jest dobrze
np.testing.assert_array_almost_equal(np.array(gen_population(5, 2, 3).shape), np.array((5, 6)))
np.testing.assert_array_almost_equal(np.array(gen_population(10, 3, 8).shape), np.array((10, 24)))

## Dekodowanie osobnika
Aby móc ocenić danego osbnika (podstawić go do funkcji celu) należy go zdekodować, czyli każdą ze zmiennych w postaci binarnej zamienić na liczbę rzeczywistą. Patrz przykład do zadania pierwszego.

### Zadanie 3
Zaimplementuj metodę pozwalajacą na rozkodowanie osobników, tzn. przekonwertowanie osobnika z postaci binarnej na rzeczywistą. Metoda powinna zwrócić jedno wymiarową tablicę `ndarray`.

Argumenty funkcji:
- `individual` - osobnik binarny kodujący `N` zmiennych rzeczywistych, tablica `ndarray`.
- `N` - liczba zmiennych, liczba całkowita.
- `B` - liczba bitów na każdą ze zmiennych, liczba całkowita.
- `a` - początek przedziału, liczba rzeczywsta, dla każdej zmiennej taki sam.
- `dx` - krok, dokładność kodowania, taki sam dla każdej zmiennej.
- `decode_individual` - rozkodowany osobnik, tablica `ndarray` zawierająca `N` zmiennych rzeczywistych.

__Ważne__: Funkcja ta wykonywana będzie w każdej iteracji algorytmu (wielokrotnie) należy zatem zadbać o to aby było ona zaimplementowana w sposób wydajny.

In [15]:

def decode_individual(individual, N, B, a, dx):
    ### TWÓJ KOD TUTAJ
    temp = individual.reshape((N, B))
    wektor= np.zeros(B)
    k=1
    b_1=B
    for i in range (0,B):
        b_1=b_1-1
        wektor[i] = 2**(b_1)
        
        
    
    kk = 0
    kkk=0
    for i in range (0,B):
        kk=(wektor[i] * temp[0][i]) + kk
    for j in range (0,B):
        kkk=(wektor[j] * temp[1][j]) + kkk
       
    wynikowy = (kk * dx) + a
    wynikowy_1 = (kkk * dx) + a
    decode_individual = np.zeros(2)
    decode_individual[0] = wynikowy
    decode_individual[1] = wynikowy_1
        
    return decode_individual  

In [170]:
aaaf = np.arange(10)
aaaf = aaaf.reshape((2,5))
print(aaaf)
type(aaaf)
wektor= np.zeros(5)
print(wektor[1])
kk=1
for i in range (0,5):
    kk=(wektor[i] * temp[0][i]) + kk
        

[[0 1 2 3 4]
 [5 6 7 8 9]]
0.0


In [178]:

wektor= np.zeros(B)
pops = np.array([0, 0, 1, 1, 1, 1, 0, 0, 0, 0]).reshape((2,5))
print(pops)

[[0 0 1 1 1]
 [1 0 0 0 0]]


In [66]:
# jeśli poniższe nie rzuca wyjątku to znaczy, że testy przeszły ale nie musi oznaczać, że wszystko jest dobrze
a = -1
N = 2
B = 5
dx = 0.06451612903225806
pop = np.array([[0, 0, 1, 1, 1, 1, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 1, 0, 0, 1], [0, 1, 0, 0, 0, 1, 1, 1, 0, 0], [1, 0, 0, 1, 1, 1, 1, 1, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 0, 1]])
dpop = np.array([[-0.5483871, 0.03225806], [-0.48387097, -0.41935484], [-0.48387097, 0.80645161], [0.22580645, 0.80645161], [1.0, 0.87096774]])
for i, ind in enumerate(pop):
    np.testing.assert_array_almost_equal(decode_individual(ind, N, B, a, dx), dpop[i])

## Ocena całej populacji
Gdy już umiemy rozkodować osobiki to możemy na każdym z nich obliczyć wartość funkcji celu.

### Zadanie 4
Zaimplementuj metodę oceny osobników w populacji, tzn. metodę wykonującą funkcję celu na każdym z osobników. Metoda powinna zwrócić jedno wymiarową tablicę `ndarray`.

Wejściem do funkcji jest populacja zakodowana, tak więc należy wykorzystać funkcję z zadania 3 do rozkodowania osobnika a następnie wywołać na nim funkcje celu.

Argumety funkcji:
- `func` - funkcja celu (przystosowania).
- `pop` - populacja zakodowanych osobników, tablica `ndarray`.
- `N` - liczba zmiennych, liczba całkowita.
- `B` - liczba bitów na każdą ze zmiennych, liczba całkowita.
- `a` - początek przedziału, liczba rzeczywsta, dla każdej zmiennej taki sam.
- `dx` - krok, dokładność kodowania, taki sam dla każdej zmiennej.
- `evaluated_pop` - tablica `ndarray` zawierająca wartości funkcji celu dla poszczególnych osobników.

__Ważne__: Funkcja ta wykonywana będzie w każdej iteracji algorytmu (wielokrotnie) należy zatem zadbać o to aby było ona zaimplementowana w sposób wydajny.

In [64]:
def evaluate_population(func, pop, N, B, a, dx):
    evaluated_pop = np.ndarray(int(pop.size/(B*N)))
    for i in range(int(pop.size/(B*N))):
        evaluated_pop[i] = func(decode_individual(pop[i], N, B, a, dx))
    return evaluated_pop
    


 
 
 


In [65]:
# jeśli poniższe nie rzuca wyjątku to znaczy, że testy przeszły ale nie musi oznaczać, że wszystko jest dobrze
a = -1
N = 2
B = 5
dx = 0.06451612903225806
pop = np.array([[0, 0, 1, 1, 1, 1, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 1, 0, 0, 1], [0, 1, 0, 0, 0, 1, 1, 1, 0, 0], [1, 0, 0, 1, 1, 1, 1, 1, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 0, 1]])
epop = np.array([0.30176899, 0.40998959, 0.88449532, 0.70135276, 1.75858481])
np.testing.assert_array_almost_equal(evaluate_population(obj_func, pop, N, B, a, dx), epop)

## Wybór najlepszego osobnika
W działaniu algorytmu genetycznego chodzi o to aby znaleźć osobnika, który ma nalepszą wartość funkcji. Będziemy implementowąć algorytm genetyczny w wersji algortmu maksymalizującego wartość funkcji celu. Zatem najlepszy osobnik to ten,którego wartość funkcji celu jest największa.

### Zadanie 5
Zaimplementować metodę zwracającą najlepszego osobnika z populacji (maksimum). Metoda powinna zwracać osobnika w postaci jednowymiarowej tablicy `ndarray` oraz odpowiadającą mu wartość funkcji celu.

- `pop` - populacja zakodowanych osobników, tablica `ndarray`.
- `evaluated_pop` - tablica `ndarray` ocen osobników.
- `best_individual` - najlepszy osobnik (zakodowany), tablica `ndarray`.
- `best_value` - wartość najlepszego osobnika, liczba rzeczywista.

__Ważne__: Funkcja ta wykonywana będzie w każdej iteracji algorytmu (wielokrotnie) należy zatem zadbać o to aby było ona zaimplementowana w sposób wydajny.

In [62]:
def get_best(pop, evaluated_pop):
    ### TWÓJ KOD TUTAJ
    best_value=evaluated_pop[0]
    for i in evaluated_pop:
        if i>best_value:   #i przyjmuje wartości kolejnych komórek tablicy tab
            best_value=i
    indeks=0    
    for j in range (0,evaluated_pop.size):
        if evaluated_pop[j] == best_value:
            indeks = j
    best_individual = np.zeros(pop.size)
    best_individual = pop[indeks]
    return best_individual, best_value

In [61]:
# jeśli poniższe nie rzuca wyjątku to znaczy, że testy przeszły ale nie musi oznaczać, że wszystko jest dobrze
pop = np.array([[0, 0, 1, 1, 1, 1, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 1, 0, 0, 1], [0, 1, 0, 0, 0, 1, 1, 1, 0, 0], [1, 0, 0, 1, 1, 1, 1, 1, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 0, 1]])
epop = np.array([0.30176899, 0.40998959, 0.88449532, 0.70135276, 1.75858481])
np.testing.assert_array_almost_equal(get_best(pop, epop)[0], np.array([1, 1, 1, 1, 1, 1, 1, 1, 0, 1]))
np.testing.assert_almost_equal(get_best(pop, epop)[1], 1.75858481)

&copy; Katedra Informatyki, Politechnika Krakowska